In [42]:
import pandas as pd
import datetime

In [43]:
import pyodbc

current_time = datetime.datetime.now()

load_year = current_time.year
load_month = current_time.month
load_day = current_time.day

landing_zone_path = f'../../Landing-Zone/Sales/Pizza/pizza_sales_{load_year}-{load_month}-{load_day}.csv'

try:
    df = pd.read_csv(landing_zone_path)
except FileNotFoundError:
    print(f"Error: The file {landing_zone_path} does not exist.")
    raise

df['src_file_name'] = landing_zone_path.split('/')[-1]
df['ingestion_date'] = current_time 

# Correct connection string
connection_string = r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-UEF1N3GJ\SQLEXPRESS;DATABASE=pizza_sales;Trusted_Connection=yes;"

try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
except pyodbc.Error as e:
    print("Error connecting to SQL Server:", e)
    raise

# Ensure the bronze schema exists
try:
    cursor.execute("IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'bronze') BEGIN CREATE SCHEMA bronze; END")
    conn.commit()
except pyodbc.Error as e:
    print("Error ensuring schema exists:", e)
    conn.rollback()

# Generate SQL for creating the table
def generate_create_table_sql(schema, table_name, df):
    type_mapping = {
        "int64": "INT",
        "float64": "FLOAT",
        "object": "VARCHAR(255)",   # Most CSV columns are strings
        "datetime64[ns]": "DATETIME",
        "date": "DATE"              # For date columns
    }

    columns = []
    for col, dtype in df.dtypes.items():
        # Special handling for ingestion_date to ensure it's DATETIME
        if col == 'ingestion_date':
            sql_type = "DATETIME"
        else:
            sql_type = type_mapping.get(str(dtype), "VARCHAR(255)")
        columns.append(f"[{col}] {sql_type}")

    columns_sql = ",\n".join(columns)

    create_table_sql = f"""
    IF OBJECT_ID('{schema}.{table_name}', 'U') IS NULL
    BEGIN
        CREATE TABLE {schema}.{table_name} (
            {columns_sql}
        )
    END
    """
    return create_table_sql

# Create the bronze.pizza_sales table
try:
    create_table_sql = generate_create_table_sql("bronze", "pizza_sales", df)
    cursor.execute(create_table_sql)
    conn.commit()
    print("Table bronze.pizza_sales created (or already exists).")
except pyodbc.Error as e:
    print("Error creating table:", e)
    conn.rollback()

# Insert data into the table
try:
    for index, row in df.iterrows():
        values = tuple(row)
        placeholders = ','.join('?' * len(row))
        sql = f"INSERT INTO bronze.pizza_sales VALUES ({placeholders})"
        cursor.execute(sql, values)

    conn.commit()
    print("Data inserted into bronze.pizza_sales.")
except pyodbc.Error as e:
    print("Error inserting data:", e)
    conn.rollback()

# Close the connection
cursor.close()
conn.close()

Error ensuring schema exists: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near the keyword 'SCHEMA'. (156) (SQLExecDirectW)")
Table bronze.pizza_sales created (or already exists).
Data inserted into bronze.pizza_sales.
